In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
import numpy as np
import tensorflow as tf
import cv2
from tqdm import tqdm_notebook
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
train_path = '/content/drive/My Drive/Face Mask Dataset/Train'
test_path = '/content/drive/My Drive/Face Mask Dataset/Test'

In [ ]:
IMG_SHAPE = (128,128,3) 
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE, include_top=False, weights='imagenet')

In [ ]:
base_model.trainable = False

In [ ]:
#CABECERA PERSONALIZADA 
Cab_Model = base_model.output
Cab_Model = tf.keras.layers.GlobalAvgPool2D()(Cab_Model)
Cab_Model = tf.keras.layers.Dense(128, activation='relu')(Cab_Model)
Cab_Model = tf.keras.layers.Dropout(0.5)(Cab_Model)
Cab_Model = tf.keras.layers.Dense(1, activation='sigmoid')(Cab_Model)

In [ ]:
model = tf.keras.models.Model(inputs= base_model.input, outputs=Cab_Model)

In [ ]:
opt = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=opt, loss="binary_crossentropy", metrics=['accuracy'])

In [ ]:
#DATA GENERATOR
train_path_gen = ImageDataGenerator(rescale=1/255.)
test_path_gen = ImageDataGenerator(rescale=1/255.)

In [ ]:
train_generator = train_path_gen.flow_from_directory(train_path, target_size=(128,128), batch_size=128, class_mode='binary')
test_generator = test_path_gen.flow_from_directory(test_path, target_size=(128,128), batch_size=128, class_mode='binary')

In [ ]:
print(train_generator.class_indices)

In [ ]:
from keras.callbacks import History
history = History()

In [ ]:
model.fit(train_generator, epochs=4, validation_data=test_generator, callbacks=[history])

In [ ]:
from keras.preprocessing import image
from keras.applications.mobilenet_v2 import preprocess_input, decode_predictions

img =  '/content/drive/My Drive/con2.png'
img = image.load_img(img, target_size=(128,128))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

preds = model.predict(x)
print(preds)
resultado = preds[0]
predic = resultado[0]
print(resultado)
if (resultado>0.001):
  print('Sin mascarilla')
else:
  print('Con mascarilla')

In [ ]:
import cv2
from google.colab.patches import cv2_imshow

In [ ]:
model.save("detec_mask_final.model2", save_format="h5")
model.save_weights("my_weights_final2", save_format="h5")

In [ ]:
from google.colab import files
files.download("my_weights_final2")